# Jon Fosse Maps

This notebook utilise data from Sceneweb about theatre productions based on plays by Jon Fosse, to create map visualisations. The objective is to visualise the geographical distribution of Fosse's work over time (1994-2024).

The script:
- loads data from Excel
- geocode addresses and place names into geo-coordinates (OpenCage's Geocoding API)
- creates an interactive map with all productions
- creates a heatmap of productions over time

## Import packages

In [19]:
import pandas as pd
import json
from geopy.geocoders import GoogleV3
import time
import folium
from folium.plugins import TimeSliderChoropleth
from folium.plugins import TimestampedGeoJson

### Load data

In [3]:
df = pd.read_excel('./data/sceneweb_fosse_productions_cleaned.xlsx')

## Geocode addresses to coordinates

In [ ]:
from opencage.geocoder import OpenCageGeocode

key = 'API-key' # Replace with your own API key
geocoder = OpenCageGeocode(key)

def geocode_address(row):
    address = f"{row['street']}, {row['postal-code']} {row['place']}, {row['country']}"
    result = geocoder.geocode(address)
    time.sleep(2)  # Ensures compliance with OpenCage's rate limit of 1 request per second
    if result:
        latitude = result[0]['geometry']['lat']
        longitude = result[0]['geometry']['lng']
        return pd.Series([latitude, longitude])
    return pd.Series([None, None])

df[['latitude', 'longitude']] = df.apply(geocode_address, axis=1)

## Convert date-premiere to datetime

In [9]:
# Convert date-premiere to datetime
df['date-premiere'] = pd.to_datetime(df['date-premiere'])

# Extract the year for grouping in the map
df['year'] = df['date-premiere'].dt.year

## Base map with pins

The code cell below creates an interactive base map with pins for all of the productions in the dataset.

You can also [preview the interactive map here](http://htmlpreview.github.io/?https://github.com/joncto/Jon-Fosse-Heatmap/blob/main/Jon-Fosse-Heatmap/output/fosse-map-pins.html).

In [10]:
# Base map (set the initial location)
map = folium.Map(location=[48.8566, 2.3522], zoom_start=5)

# Prepare data
data = df[['latitude', 'longitude', 'year', 'title', 'place']].dropna()

# Function to create popup
def make_popup(row):
    return folium.Popup(f"{row['title']} at {row['place']}, {row['year']}")

# Plot each point
for idx, row in data.iterrows():
    folium.Marker(
        [row['latitude'], row['longitude']],
        popup=make_popup(row)
    ).add_to(map)

# Save or display the map
map.save('./output/fosse-map-pins.html')


## Heatmap with time

The code cell below creates a heatmap visualising the distribution of productions over time (per year).

You can also [preview the heatmap here](http://htmlpreview.github.io/?https://github.com/joncto/Jon-Fosse-Heatmap/blob/main/Jon-Fosse-Heatmap/output/fosse-heatmap.html).

In [15]:
import folium
from folium.plugins import HeatMapWithTime

# Base map
m = folium.Map(location=[45.8566, 0.3522], zoom_start=2)  # You can adjust the initial location

# Prepare data for the HeatMapWithTime
data_by_year = []
years = sorted(df['year'].unique())

for year in years:
    year_data = df[df['year'] == year][['latitude', 'longitude']].dropna()
    data_by_year.append(year_data.values.tolist())  # Each entry is a list of [lat, lng]

# Create a HeatMapWithTime
heatmap = HeatMapWithTime(data_by_year, index=years, auto_play=True, max_opacity=0.8)

# Add it to the base map
heatmap.add_to(m)

# Save the map to html (which can be opened in a web browser)
m.save('./output/animated_heatmap.html')